# 爬虫南京地铁

In [1]:
import requests
import time
import json
import ast
import os
import utils
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import etree

In [2]:
url="http://map.amap.com/subway/index.html?&1100"
header={"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}
res=requests.get(url,header)
print(res)
DATA_URL = 'http://map.amap.com/service/subway?_1579353166335&srhdata='
http://map.amap.com/service/subway?_1579353166335&srhdata=3201_info_nanjing.json

<Response [200]>


In [12]:
def Fetch_All_City(url,header):
    r =requests.get(url,header)
    html = r.content
    element = etree.HTML(html)
    options = element.xpath("//a[contains(@class,'city')]")
    cities = []
    for option in options:
        city = {
            'id': option.get('id'),
            'name': option.get('cityname'),
            'text': option.text
        }
        cities.append(city)
        #print(city)
    print(cities)
Fetch_All_City(url,header)


[{'id': '1100', 'name': 'beijing', 'text': '北京'}, {'id': '3100', 'name': 'shanghai', 'text': '上海'}, {'id': '4401', 'name': 'guangzhou', 'text': '广州'}, {'id': '4403', 'name': 'shenzhen', 'text': '深圳'}, {'id': '4201', 'name': 'wuhan', 'text': '武汉'}, {'id': '1200', 'name': 'tianjin', 'text': '天津'}, {'id': '3201', 'name': 'nanjing', 'text': '南京'}, {'id': '8100', 'name': 'xianggang', 'text': '香港'}, {'id': '5000', 'name': 'chongqing', 'text': '重庆'}, {'id': '3301', 'name': 'hangzhou', 'text': '杭州'}, {'id': '2101', 'name': 'shenyang', 'text': '沈阳'}, {'id': '2102', 'name': 'dalian', 'text': '大连'}, {'id': '5101', 'name': 'chengdu', 'text': '成都'}, {'id': '2201', 'name': 'changchun', 'text': '长春'}, {'id': '3205', 'name': 'suzhou', 'text': '苏州'}, {'id': '4406', 'name': 'foshan', 'text': '佛山'}, {'id': '5301', 'name': 'kunming', 'text': '昆明'}, {'id': '6101', 'name': 'xian', 'text': '西安'}, {'id': '4101', 'name': 'zhengzhou', 'text': '郑州'}, {'id': '4301', 'name': 'changsha', 'text': '长沙'}, {'id': '3302

In [13]:
def Parse_All_CityData(cities):
    browser = webdriver.Chrome()
    browser.get(url)
    Data = []
    for city in cities:
        Data.append(Parse_CityData(city,browser))
    return data

In [11]:
def Parse_CityData_FromApi(city):    
    url_cityname= DATA_URL + "{}_drw_{}.json".format(city['id'],city['name'])
    print(url_cityname)
    r = requests.get(url_cityname)
    return  eval(r.text.encode('utf-8'))# 字符串转json(ast.literal_eval 

TypeError: string indices must be integers

In [ ]:
def Parse_CityData_FromDom(city,broser):
    menu = browser.find_element_by_css_selector(".more-city")
    el   = browser.find_element_by_id(id)
    webdriver.common.action_chains.ActionChains(browser).move_to_element(menu).click(e1).perform()
    time.sleep(2)
    element = etree.HTML(browser.page_source)
    station_names = element.xpath("//g[@id='g-station-name']/g/text")
    result = {
        'st': {}
    }

    for station in station_names:
        sid = station.get("id").split('-')[1]
        result['st'][sid]={}
        result['st'][sid]['lp']={
            'x': int(station.get('x')),
            'y': int(station.get('y'))
        }
    return result

In [6]:
def format_CityData(apiData, domData):
   # string to json 
   data = {
        'name': apiData['s'],
        'l': [],
   }
   lines = apiData['l']
   # list get index and value
   for lidx, line in enumerate(lines):
        labelp = []
        p = []
        for val in line['lp']:
            labelp.append(utils.getP(val))
        for val in line['c']:
            p.append(utils.getP(val))
        l  = {
            'name': line['ln'],
            'p': p,
            'labelp': labelp,
            'st': [],
            'la': line['la']
        }
        for sidx, stop in enumerate(line['st']):
           # 开通的站
           if line['st'][sidx]['su'] == '1':
                labelp  = domData['st'][stop['si']]['lp'];
                st = {
                    'name': stop['n'],
                    'p': utils.getP(stop['p']),
                    'labelp': labelp,
                    'strans': bool(int(stop['t'])),
                    'ldir': utils.getLdir(stop['lg']),
                }
                l['st'].append(st)
        data["l"].append(l)

   return data

In [7]:
def Parse_CityData(city,browser):    
    apiData = Parse_CityData_FromApi(city)
    domData = Parse_CityData_FromDom(city, browser)    
    return    format_CityData(apiData, domData)

In [8]:
def main():
    cities = Fetch_All_City(url, header)
    print(cities)
    citiesData = Parse_All_CityData(cities)
    saveData(citiesData)

if __name__ == '__main__':
    main()

[{'id': '1100', 'name': 'beijing', 'text': '北京'}, {'id': '3100', 'name': 'shanghai', 'text': '上海'}, {'id': '4401', 'name': 'guangzhou', 'text': '广州'}, {'id': '4403', 'name': 'shenzhen', 'text': '深圳'}, {'id': '4201', 'name': 'wuhan', 'text': '武汉'}, {'id': '1200', 'name': 'tianjin', 'text': '天津'}, {'id': '3201', 'name': 'nanjing', 'text': '南京'}, {'id': '8100', 'name': 'xianggang', 'text': '香港'}, {'id': '5000', 'name': 'chongqing', 'text': '重庆'}, {'id': '3301', 'name': 'hangzhou', 'text': '杭州'}, {'id': '2101', 'name': 'shenyang', 'text': '沈阳'}, {'id': '2102', 'name': 'dalian', 'text': '大连'}, {'id': '5101', 'name': 'chengdu', 'text': '成都'}, {'id': '2201', 'name': 'changchun', 'text': '长春'}, {'id': '3205', 'name': 'suzhou', 'text': '苏州'}, {'id': '4406', 'name': 'foshan', 'text': '佛山'}, {'id': '5301', 'name': 'kunming', 'text': '昆明'}, {'id': '6101', 'name': 'xian', 'text': '西安'}, {'id': '4101', 'name': 'zhengzhou', 'text': '郑州'}, {'id': '4301', 'name': 'changsha', 'text': '长沙'}, {'id': '3302

TypeError: 'NoneType' object is not iterable